In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

In [39]:
# Dates and other constants
# Define path to files
# For MacOS
PATH = "/Users/Mustapha/Desktop/New_folder_(2)/Tsa_practice/stocks/Stock_list/"
PATH2 = "/Users/Mustapha/Desktop/New_folder_(2)/Tsa_practice/stocks/Stock_list/stocklist_updated/"

# For Windows
# PATH = "D:/Python for Finance/Stocks/"

# Start date defaults
S_YEAR = 2021
S_MONTH = 1
S_DAY = 31
S_DATE_STR = "2021-01-01"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2022
E_MONTH = 7
E_DAY = 1
E_DATE_STR = "2022-01-01"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

In [16]:
# Function that gets a dataframe by providing a ticker and starting date
# saves the data downloaded as csv from yahoo
def save_to_csv_from_yahoo(ticker, S_YEAR, S_MONTH, S_DAY, E_YEAR, E_MONTH, E_DAY):
    
    # Defines the time periods to use
    start = dt.datetime(S_YEAR, S_MONTH, S_DAY)
    end = dt.datetime( E_YEAR, E_MONTH, E_DAY)
    
    # Reads data into a dataframe
    df = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
    
    # Save data to a CSV file
    # For Windows
    # df.to_csv('C:/Users/derek/Documents/Python Finance/Python/' + ticker + '.csv')
    # For MacOS
    df.to_csv("/Users/Mustapha/Desktop/New_folder_(2)/Tsa_practice/stocks/Stock_list/" + ticker + '.csv')
    return df

In [27]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

In [17]:
def download_multiple_stocks(S_YEAR, S_MONTH, S_DAY, E_YEAR, E_MONTH, E_DAY, *args):
    for x in args:
        save_to_csv_from_yahoo(x, S_YEAR, S_MONTH, S_DAY, E_YEAR, E_MONTH, E_DAY)

In [18]:
# Stocks downloaded 
# JNJ ; Johnson and Johnson
# AVT ; Avnet
# TSLA ; Tesla
# VZ ; Verizon Communication
# KMB ; Kimberly Clark

# args = ["JNJ", "AVT", "TSLA", "VZ", "KMB"]
# mult_df = download_multiple_stocks(2021, 1, 1, 2022, 7, 1, *tickers)
# Download completed and saved in path provided

In [29]:
# Add daily returns to dataframe
# We calculate a percentage rate of return for each day to compare investments.
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1

# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    df.to_csv(PATH2 + ticker + '.csv')
    return df  

In [22]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [30]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH2 + ticker + '.csv')

In [124]:
test = get_df_from_csv('TSLA')

,Date,Adj Close
0,2021-01-04,729.770020
1,2021-01-05,735.109985
2,2021-01-06,755.979980
3,2021-01-07,816.039978
4,2021-01-08,880.020020
...,...,...
372,2022-06-27,734.760010
373,2022-06-28,697.989990
374,2022-06-29,685.469971
375,2022-06-30,673.419983


In [40]:
for arg in args:
    print('Working on :', arg)
    print('Completed')
    # Get a dataframe for that ticker
    stock_df = get_df_from_csv(arg)
    
    # Add daily return to this dataframe
    add_daily_return_to_df(stock_df, arg)
    
    # Delete unnamed columns in dataframe
    stock_df = delete_unnamed_cols(stock_df)
    
    # Save cleaned dataframe to csv
    save_dataframe_to_csv(stock_df, arg)

Working on : JNJ
Completed
Working on : AVT
Completed
Working on : TSLA
Completed
Working on : VZ
Completed
Working on : KMB
Completed


In [51]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv_2(ticker):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv("/Users/Mustapha/Desktop/New_folder_(2)/Tsa_practice/stocks/Stock_list/stocklist_updated/" + ticker + '.csv')    
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df
    
    

In [73]:
def merge_df_by_column_name(col_name, S_YEAR, S_MONTH, S_DAY, E_YEAR, E_MONTH, E_DAY, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    start = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
    end = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
    
    for x in tickers:
        df = get_df_from_csv_2(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= start) & (df['Date'] <= end)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

In [74]:
portfolio_list = ["JNJ", "AVT", "TSLA", "VZ", "KMB"]

In [121]:
multiple_df = merge_df_by_column_name('daily_return',  2021, 1, 1, 2022, 7, 1, *portfolio_list)
# multiple_df
multiple_df.corr() # correlation


,JNJ,AVT,TSLA,VZ,KMB
JNJ,1.000000,0.158157,0.008618,0.375776,0.441621
AVT,0.158157,1.000000,0.323848,0.128724,0.050785
TSLA,0.008618,0.323848,1.000000,-0.048070,-0.045542
VZ,0.375776,0.128724,-0.048070,1.000000,0.281687
KMB,0.441621,0.050785,-0.045542,0.281687,1.000000


In [77]:
# Get the number of samples
days = len(multiple_df.index)
print('The total number of days present in the dataframe is', days)

The total number of days present in the dataframe is 377


In [78]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv_2(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [82]:
get_stock_price_on_date('TSLA', '2022-02-04')

923.3200073242188

In [95]:
# 01-04-2021 is the 1st date in 2021
print("JNJ $", get_stock_price_on_date('JNJ', '2021-01-04')) # 150.7
print("AVT $", get_stock_price_on_date('AVT', '2021-01-04')) # 34.17
print("TSLA $", get_stock_price_on_date('TSLA', '2021-01-04')) # 729.77
print("VZ $", get_stock_price_on_date('VZ', '2021-01-04')) # 1 54.28
print("KMB $", get_stock_price_on_date('KMB', '2021-01-04')) # 1 126.66


# Total Investment
tot_inv = 150.7 + 34.17 + 729.77 + 54.28 + 126.66
tot_inv

JNJ $ 150.71075439453125
AVT $ 34.1702995300293
TSLA $ 729.77001953125
VZ $ 54.27604293823242
KMB $ 126.65752410888672


1095.58

In [96]:
# Returns portfolio weight
def get_port_weight(price, total):
    return price / total

In [97]:
jnj_w = get_port_weight(150.71, 1095.58)
avt_w = get_port_weight(34.17, 1095.58)
tsla_w = get_port_weight(729.77, 1095.58)
vz_w = get_port_weight(54.28, 1095.58)
kmb_w = get_port_weight(126.66, 1095.58)

In [111]:
weights = np.array([jnj_w, avt_w, tsla_w, vz_w, kmb_w])
# weights
weights.sum() #The sum of porfolio weights is equal to 1

1.0000091275853886

In [108]:
port_var = np.dot(weights, np.dot(multiple_df.cov() * 377, weights))
print("Portfolio Covariance :", port_var)
print("JNJ Var :", multiple_df["JNJ"].var() * 377)
print("AVT Var :", multiple_df["AVT"].var() * 377)
print("TSLA Var :", multiple_df["TSLA"].var() * 377)
print("VZ Var :", multiple_df["VZ"].var() * 377)
print("KMB Var :", multiple_df["KMB"].var() * 377)

Portfolio Covariance : 0.25409701281821917
JNJ Var : 0.039089631998379254
AVT Var : 0.13059020923873857
TSLA Var : 0.5612519834029792
VZ Var : 0.04289636808847164
KMB Var : 0.05884018709666931


In [125]:
# multiple_df

In [117]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(multiple_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = multiple_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [119]:
diversifiable_risk = calc_diversifiable_risk(multiple_df, portfolio_list, weights)
diversifiable_risk

0.0033142535758332085

In [120]:
# Risk that cannot be diversified ; Systematic Risk
print("Systematic Risk :", (port_var - diversifiable_risk))


Systematic Risk : 0.25078275924238597
